In [ ]:
#!/usr/bin/env python3
"""
UCRAG Website Server
Run this file with: python ucrag_server.py
Then open http://localhost:8080 in your browser
"""

from http.server import HTTPServer, BaseHTTPRequestHandler
import json
from urllib.parse import parse_qs, urlparse

# Store events in memory (resets when server restarts)
events = []

HTML_CONTENT = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>UCRAG - Unnecessarily Competitive Recreational Activities Group</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.6;
            color: #333;
            background-color: #f4f4f4;
        }

        header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 1.5rem 2rem;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }

        .logo-container {
            display: flex;
            align-items: center;
            gap: 1rem;
            margin-bottom: 1rem;
        }

        .logo {
            height: 60px;
            width: 60px;
            background: white;
            padding: 10px;
            border-radius: 10px;
            font-size: 24px;
            font-weight: bold;
            text-align: center;
            line-height: 40px;
            color: #667eea;
        }

        .logo-container h1 {
            font-size: 2.5rem;
            font-weight: bold;
        }

        nav {
            display: flex;
            gap: 1rem;
            flex-wrap: wrap;
        }

        .tab-btn {
            background: rgba(255,255,255,0.2);
            color: white;
            border: 2px solid white;
            padding: 0.7rem 1.5rem;
            font-size: 1rem;
            cursor: pointer;
            border-radius: 5px;
            transition: all 0.3s;
        }

        .tab-btn:hover, .tab-btn.active {
            background: white;
            color: #667eea;
        }

        .admin-btn {
            background: rgba(255,255,255,0.1);
            color: white;
            border: 1px solid rgba(255,255,255,0.5);
            padding: 0.7rem 1.5rem;
            font-size: 0.9rem;
            cursor: pointer;
            border-radius: 5px;
            margin-left: auto;
            transition: all 0.3s;
        }

        .admin-btn:hover {
            background: rgba(255,255,255,0.3);
        }

        main {
            max-width: 1200px;
            margin: 2rem auto;
            padding: 0 2rem;
            min-height: calc(100vh - 300px);
        }

        .tab-content {
            display: none;
            animation: fadeIn 0.5s;
        }

        .tab-content.active {
            display: block;
        }

        @keyframes fadeIn {
            from { opacity: 0; }
            to { opacity: 1; }
        }

        .about-container {
            background: white;
            padding: 3rem;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }

        .about-text h2 {
            color: #667eea;
            font-size: 2rem;
            margin-bottom: 1.5rem;
        }

        .mission-statement {
            font-size: 1.2rem;
            line-height: 1.8;
            color: #555;
            margin-bottom: 2rem;
            text-align: justify;
        }

        .about-images {
            display: flex;
            gap: 2rem;
            justify-content: center;
            flex-wrap: wrap;
        }

        .activity-img {
            width: 300px;
            height: 300px;
            object-fit: cover;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
            transition: transform 0.3s;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            display: flex;
            align-items: center;
            justify-content: center;
            color: white;
            font-size: 1.5rem;
            font-weight: bold;
            text-align: center;
            padding: 2rem;
        }

        .activity-img:hover {
            transform: scale(1.05);
        }

        #events-list {
            background: white;
            padding: 2rem;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }

        .no-events {
            text-align: center;
            color: #999;
            font-size: 1.2rem;
            padding: 3rem;
        }

        .event-card {
            background: #f9f9f9;
            padding: 1.5rem;
            margin-bottom: 1.5rem;
            border-radius: 8px;
            border-left: 5px solid #667eea;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }

        .event-card h3 {
            color: #667eea;
            margin-bottom: 0.5rem;
        }

        .event-card .event-details {
            color: #666;
            font-size: 0.95rem;
            margin-bottom: 0.5rem;
        }

        .event-card .event-description {
            color: #555;
            line-height: 1.6;
        }

        .admin-panel {
            display: none;
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0,0,0,0.7);
            z-index: 1000;
            overflow-y: auto;
        }

        .admin-panel.active {
            display: flex;
            justify-content: center;
            align-items: center;
            padding: 2rem;
        }

        .admin-content {
            background: white;
            padding: 2rem;
            border-radius: 10px;
            max-width: 600px;
            width: 100%;
            position: relative;
        }

        .close-btn {
            position: absolute;
            top: 1rem;
            right: 1rem;
            background: none;
            border: none;
            font-size: 2rem;
            cursor: pointer;
            color: #999;
        }

        .close-btn:hover {
            color: #333;
        }

        .admin-form {
            margin-bottom: 2rem;
        }

        .admin-form h4 {
            color: #667eea;
            margin-bottom: 1rem;
        }

        .admin-form input,
        .admin-form textarea {
            width: 100%;
            padding: 0.7rem;
            margin-bottom: 1rem;
            border: 1px solid #ddd;
            border-radius: 5px;
            font-family: inherit;
        }

        .admin-form button {
            background: #667eea;
            color: white;
            border: none;
            padding: 0.8rem 2rem;
            border-radius: 5px;
            cursor: pointer;
            font-size: 1rem;
            transition: background 0.3s;
        }

        .admin-form button:hover {
            background: #5568d3;
        }

        .admin-events h4 {
            color: #667eea;
            margin-bottom: 1rem;
        }

        .admin-event-item {
            background: #f9f9f9;
            padding: 1rem;
            margin-bottom: 1rem;
            border-radius: 5px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }

        .delete-btn {
            background: #e74c3c;
            color: white;
            border: none;
            padding: 0.5rem 1rem;
            border-radius: 5px;
            cursor: pointer;
        }

        .delete-btn:hover {
            background: #c0392b;
        }

        footer {
            background: #333;
            color: white;
            text-align: center;
            padding: 2rem;
            margin-top: 3rem;
        }

        @media (max-width: 768px) {
            .about-container {
                padding: 1.5rem;
            }

            .activity-img {
                width: 100%;
                max-width: 300px;
            }

            .logo-container h1 {
                font-size: 1.8rem;
            }
        }
    </style>
</head>
<body>
    <header>
        <div class="logo-container">
            <div class="logo">🎯</div>
            <h1>UCRAG</h1>
        </div>
        <nav>
            <button class="tab-btn active" onclick="openTab('about')">About</button>
            <button class="tab-btn" onclick="openTab('events')">Events</button>
            <button class="admin-btn" onclick="toggleAdmin()">Admin Login</button>
        </nav>
    </header>

    <main>
        <!-- About Tab -->
        <div id="about" class="tab-content active">
            <div class="about-container">
                <div class="about-text">
                    <h2>Welcome to UCRAG</h2>
                    <p class="mission-statement">
                        So many people in the world are so good at recreational games, but never get to showcase their skills or have real competition with other recreational games enthusiasts. I founded UCRAG with the goal to ensure that everyone can have a competitive outlet for recreational activities, whether that be chess, catan, cornhole, joust, or other games that are so fun but have no competition. Find your nerdy and unnecessarily competitive recreational people in this community, with events hosted monthly in the San Francisco bay area.
                    </p>
                </div>
                <div class="about-images">
                    <div class="activity-img">♟️<br>Chess</div>
                    <div class="activity-img">🎯<br>Cornhole</div>
                </div>
            </div>
        </div>

        <!-- Events Tab -->
        <div id="events" class="tab-content">
            <h2>Upcoming Events</h2>
            <div id="events-list">
                <p class="no-events">No events planned</p>
            </div>
        </div>

        <!-- Admin Panel -->
        <div id="admin-panel" class="admin-panel">
            <div class="admin-content">
                <h3>Admin Panel</h3>
                <button class="close-btn" onclick="toggleAdmin()">×</button>

                <div class="admin-form">
                    <h4>Add New Event</h4>
                    <input type="text" id="event-title" placeholder="Event Title" />
                    <input type="date" id="event-date" />
                    <input type="time" id="event-time" />
                    <input type="text" id="event-location" placeholder="Location" />
                    <textarea id="event-description" placeholder="Event Description" rows="4"></textarea>
                    <button onclick="addEvent()">Add Event</button>
                </div>

                <div class="admin-events">
                    <h4>Manage Events</h4>
                    <div id="admin-events-list"></div>
                </div>
            </div>
        </div>
    </main>

    <footer>
        <p>&copy; 2026 UCRAG - Unnecessarily Competitive Recreational Activities Group | San Francisco Bay Area</p>
    </footer>

    <script>
        async function loadEvents() {
            const response = await fetch('/api/events');
            const events = await response.json();
            displayEvents(events);
        }

        function openTab(tabName) {
            const tabs = document.querySelectorAll('.tab-content');
            const buttons = document.querySelectorAll('.tab-btn');

            tabs.forEach(tab => tab.classList.remove('active'));
            buttons.forEach(btn => btn.classList.remove('active'));

            document.getElementById(tabName).classList.add('active');
            event.target.classList.add('active');

            if (tabName === 'events') {
                loadEvents();
            }
        }

        function toggleAdmin() {
            const panel = document.getElementById('admin-panel');
            panel.classList.toggle('active');

            if (panel.classList.contains('active')) {
                loadAdminEvents();
            }
        }

        async function addEvent() {
            const title = document.getElementById('event-title').value;
            const date = document.getElementById('event-date').value;
            const time = document.getElementById('event-time').value;
            const location = document.getElementById('event-location').value;
            const description = document.getElementById('event-description').value;

            if (!title || !date || !time || !location) {
                alert('Please fill in all required fields');
                return;
            }

            const newEvent = { title, date, time, location, description };

            const response = await fetch('/api/events', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify(newEvent)
            });

            if (response.ok) {
                document.getElementById('event-title').value = '';
                document.getElementById('event-date').value = '';
                document.getElementById('event-time').value = '';
                document.getElementById('event-location').value = '';
                document.getElementById('event-description').value = '';

                loadAdminEvents();
                loadEvents();
                alert('Event added successfully!');
            }
        }

        async function deleteEvent(id) {
            if (confirm('Are you sure you want to delete this event?')) {
                await fetch(`/api/events/${id}`, { method: 'DELETE' });
                loadAdminEvents();
                loadEvents();
            }
        }

        async function loadAdminEvents() {
            const response = await fetch('/api/events');
            const events = await response.json();

            const adminList = document.getElementById('admin-events-list');

            if (events.length === 0) {
                adminList.innerHTML = '<p style="color: #999;">No events to manage</p>';
                return;
            }

            adminList.innerHTML = events.map(event => `
                <div class="admin-event-item">
                    <div>
                        <strong>${event.title}</strong><br>
                        <small>${formatDate(event.date)} at ${formatTime(event.time)}</small>
                    </div>
                    <button class="delete-btn" onclick="deleteEvent(${event.id})">Delete</button>
                </div>
            `).join('');
        }

        function displayEvents(events) {
            const eventsList = document.getElementById('events-list');

            if (events.length === 0) {
                eventsList.innerHTML = '<p class="no-events">No events planned</p>';
                return;
            }

            eventsList.innerHTML = events.map(event => `
                <div class="event-card">
                    <h3>${event.title}</h3>
                    <div class="event-details">
                        <strong>📅 Date:</strong> ${formatDate(event.date)}<br>
                        <strong>🕒 Time:</strong> ${formatTime(event.time)}<br>
                        <strong>📍 Location:</strong> ${event.location}
                    </div>
                    <p class="event-description">${event.description}</p>
                </div>
            `).join('');
        }

        function formatDate(dateString) {
            const date = new Date(dateString + 'T00:00:00');
            return date.toLocaleDateString('en-US', { 
                weekday: 'long', 
                year: 'numeric', 
                month: 'long', 
                day: 'numeric' 
            });
        }

        function formatTime(timeString) {
            const [hours, minutes] = timeString.split(':');
            const date = new Date();
            date.setHours(hours, minutes);
            return date.toLocaleTimeString('en-US', { 
                hour: 'numeric', 
                minute: '2-digit',
                hour12: true 
            });
        }

        window.onload = function() {
            loadEvents();
        };
    </script>
</body>
</html>
"""

class UCRAGHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            self.wfile.write(HTML_CONTENT.encode())

        elif self.path == '/api/events':
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(events).encode())

        else:
            self.send_response(404)
            self.end_headers()

    def do_POST(self):
        if self.path == '/api/events':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            event_data = json.loads(post_data.decode())

            event_data['id'] = len(events) + 1
            events.append(event_data)
            events.sort(key=lambda x: x['date'])

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'status': 'success'}).encode())
        else:
            self.send_response(404)
            self.end_headers()

    def do_DELETE(self):
        if self.path.startswith('/api/events/'):
            event_id = int(self.path.split('/')[-1])
            global events
            events = [e for e in events if e['id'] != event_id]

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'status': 'success'}).encode())
        else:
            self.send_response(404)
            self.end_headers()

    def log_message(self, format, *args):
        print(f"[{self.date_time_string()}] {format % args}")

def run_server(port=8080):
    server_address = ('', port)
    httpd = HTTPServer(server_address, UCRAGHandler)
    print(f"\n🚀 UCRAG Server is running!")
    print(f"📡 Open your browser and go to: http://localhost:{port}")
    print(f"🛑 Press Ctrl+C to stop the server\n")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\n\n🛑 Server stopped")
        httpd.shutdown()

if __name__ == '__main__':
    run_server()


In [1]:
import os
os.getcwd()


'/Users/henryrobertson2'

In [3]:
import os
print(os.getcwd())
print(os.listdir())


/Users/henryrobertson2
['.config', 'Music', 'latin-social-vocabulary', '.condarc', '.DS_Store', 'requirements.txt', '.CFUserTextEncoding', '.xonshrc', 'UCRAG.ipynb', 'anaconda_projects', '.zshrc', 'Pictures', 'folder', 'the', '.zsh_history', '.ipython', 'Desktop', 'Library', '.matplotlib', '.lesshst', '.spyder-py3', '#', '.cups', 'Public', '.tcshrc', 'makes', '.virtual_documents', '.anaconda', 'Movies', 'Applications', '.Trash', '.ipynb_checkpoints', '.jupyter', 'Documents', '.bash_profile', 'Downloads', '.continuum', '.gitconfig', '.zsh_sessions', '.conda']
